In [1]:
# armazenar os documentos totais em variavel

import os
# rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/20_newsgroup'
rootdir = '/media/developer/DATA/doutorado/rec informacao/20_newsgroup'

docs = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #if os.path.basename(subdir) in ['alt.atheism', 'rec.sport.baseball']:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [2]:
# baixando as libs para pre processamento

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

#stemmer = PorterStemmer()
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# pre processamento

    # Remover caracteres especiais e números
    #text = re.sub('[^A-Za-z\s\']+', '', text)

def preprocess_text(text):
    # Converter tudo para letras minúsculas
    text = text.lower()
    
    # Remover as tags HTML
    text = re.sub('<[^<]+?>', '', text)
    

    
    # Tokenizar o texto em palavras
    words = nltk.word_tokenize(text)

    # remover palavras que so possua caractere especial
    words = [word for word in words if not re.match('^[^A-Za-z0-9]+$', word)]

    # remover palavras que tenha tamanho 1
    words = [word for word in words if len(word) > 1]
    
    # Remover as stopwords
    words = [word for word in words if word not in stopwords]
    
    # Aplicar o stemming
    #words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Juntar as palavras em uma única string
    text = ' '.join(words)

    
    return text



In [4]:
# rodar o pre processamento

def preprocess_collection(docs):

    preprocessed_documents = []

    for document in docs:
        tema = document['tema']
        nome_arquivo = document['nome_arquivo']
        conteudo = document['conteudo']
        
        preprocessed_conteudo = preprocess_text(conteudo)
        
        preprocessed_document = {
            'tema': tema,
            'nome_arquivo': nome_arquivo,
            'conteudo': preprocessed_conteudo
        }
        
        preprocessed_documents.append(preprocessed_document)
        
    return preprocessed_documents

In [7]:
preprocessed_documents = preprocess_collection(docs)

In [8]:
# armazenar os documentos totais em variavel

import os
#rootdir = '/media/r2-d2/E0C494CDC494A6F6/doutorado/mini_newsgroups'
rootdir = '/media/developer/DATA/doutorado/rec informacao/mini_newsgroups'

docs_consultas = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(os.path.join(subdir, file), encoding='cp1252') as f:                
            contents = f.read()
            docs_consultas.append({ 'tema': os.path.basename(subdir), 'nome_arquivo': file, 'conteudo': contents  })

In [9]:
doc_consultas_preproc = preprocess_collection(docs_consultas)

In [10]:
doc_consultas_preproc[150]

{'tema': 'comp.graphics',
 'nome_arquivo': '39008',
 'conteudo': 'xref cantaloupe.srv.cs.cmu.edu alt.binaries.pictures.supermodels:949 alt.graphics.pixutils:5400 comp.graphics:39008 alt.binaries.pictures.erotica.d:15194 alt.binaries.pictures.d:8920 path cantaloupe.srv.cs.cmu.edu magnesium.club.cc.cmu.edu news.sei.cmu.edu cis.ohio-state.edu zaphod.mps.ohio-state.edu usc elroy.jpl.nasa.gov cougar ressler ressler cougar.jpl.nasa.gov mike ressler newsgroups alt.binaries.pictures.supermodels alt.graphics.pixutils comp.graphics alt.binaries.pictures.erotica.d alt.binaries.pictures.d subject stop argument xv-3.00 24-bit oh make date 30 apr 1993 20:31:21 gmt organization jet propulsion lab mid-infrared camera project line 13 message-id reference nntp-posting-host cougar.jpl.nasa.gov stop stop stop stop argument getting silly xv-3.00 handle 24-bit image without quantizing argument moot null void please stop quibbling obsolete version program besides generate ton verbiage new shareware licensing

In [11]:
# Função para calcular o TF (Term Frequency) de um termo em um documento
def calcular_tf(termo, documento):
    palavras_documento = documento.lower().split()
    frequencia = palavras_documento.count(termo.lower())
    return frequencia / len(palavras_documento)

In [12]:
import math

# Função para calcular o IDF (Inverse Document Frequency) de um termo em uma coleção de documentos
def calcular_idf(termo, documentos):
    num_documentos_com_termo = sum(1 for documento in documentos if termo.lower() in documento.lower())
    return math.log(len(documentos) / (1 + num_documentos_com_termo))


In [13]:
# Função para calcular o TF-IDF de um termo em um documento
def calcular_tfidf(termo, documento, documentos):
    tf = calcular_tf(termo, documento)
    idf = calcular_idf(termo, documentos)
    return tf * idf

In [14]:
# Função para calcular a similaridade de cosseno entre dois documentos
def calcular_similaridade_cosseno(vetor1, vetor2):
    numerador = sum(a * b for a, b in zip(vetor1, vetor2))
    denominador = math.sqrt(sum(a * a for a in vetor1)) * math.sqrt(sum(b * b for b in vetor2))
    return numerador / denominador


In [15]:
# vetorizando

temas = [doc["tema"] for doc in preprocessed_documents]
conteudos = [doc["conteudo"] for doc in preprocessed_documents]

# Construir o vocabulário
vocabulario = set()
for documento in conteudos:
    palavras = documento.lower().split()
    vocabulario.update(palavras)

vocabulario = list(vocabulario)
vocabulario.sort()

# Calcular o TF-IDF para cada documento
tfidf_matrix = []
for documento in conteudos:
    vetor_tfidf = [calcular_tfidf(termo, documento, conteudos) for termo in vocabulario]
    tfidf_matrix.append(vetor_tfidf)

In [ ]:
input_documento = "Texto do seu documento"

# Calcular o vetor TF-IDF para o documento de entrada
input_vetor_tfidf = [calcular_tfidf(termo, input_documento, conteudos) for termo in vocabulario]

# Calcular a similaridade de cosseno entre o documento de entrada e cada documento existente
similaridades = [calcular_similaridade_cosseno(input_vetor_tfidf, vetor_tfidf) for vetor_tfidf in tfidf_matrix]

resultados = list(zip(similaridades, temas, conteudos))
resultados.sort(reverse=True)

for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
    print(f"Rank {rank}: Similaridade {similaridade:.4f} - Tema: {tema}")
    print(f"Conteúdo: {conteudo}")
    print()

In [157]:
dirs_abs = []
for subdir, dirs, files in os.walk(rootdir):
    if os.path.basename(subdir) != '20_newsgroup':
        dirs_abs.append(os.path.basename(subdir))

result_final = {}
for d in dirs_abs:
    result_final[d] = {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}

print(result_final)    

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'alt.atheism': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.graphics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.os.ms-windows.misc': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.ibm.pc.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.sys.mac.hardware': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'comp.windows.x': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'misc.forsale': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.autos': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.motorcycles': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.baseball': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'rec.sport.hockey': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.crypt': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0}, 'sci.electronics': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 

In [146]:
def calcula_map(resultados, tema_verificado):
    acertos = 0
    maps = []
    #for i, doc in enumerate(ranking):
    for i, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if tema_verificado == tema:
            acertos +=1
            maps.append(acertos/(i+1))
    return mean(maps)

In [160]:
from statistics import mean

#input_documento = doc_consultas_preproc[150]['conteudo']

p10 = []
p20 = []
p50 = []
p100 = []
map_f = []
tema_atual = ""
for c, input_documento in enumerate(doc_consultas_preproc):
    
    if c == 0:
        tema_atual = input_documento['tema']
    
    acertos = 0     

    input_tfidf = vectorizer.transform([input_documento['conteudo']])
    similaridades = cosine_similarity(input_tfidf, tfidf_matrix)[0]

    resultados = list(zip(similaridades, temas, conteudos))
    resultados.sort(reverse=True)

    map_f.append(calcula_map(resultados, input_documento['tema']))
    for rank, (similaridade, tema, conteudo) in enumerate(resultados, start=1):
        if input_documento['tema'] == tema:
            acertos += 1
            
        if rank == 10:
            p10.append(acertos/10.0)
        if rank == 20:
            p20.append(acertos/20.0)
        if rank == 50:
            p50.append(acertos/50.0)
        if rank == 100:
            p100.append(acertos/100.0)
        
    
    if tema_atual != input_documento['tema'] or c == len(doc_consultas_preproc) - 1:
        result_final[tema_atual]['p10'] = mean(p10)
        result_final[tema_atual]['p20'] = mean(p20)
        result_final[tema_atual]['p50'] = mean(p50)
        result_final[tema_atual]['p100'] = mean(p100)     
        result_final[tema_atual]['map'] = mean(map_f)
        p10 = []
        p20 = []
        p50 = []
        p100 = []
        map_f = []           

    tema_atual = input_documento['tema']        

    if c > 102:
        break    
    


In [161]:
result_final

{'mini_newsgroups': {'p10': 0, 'p20': 0, 'p50': 0, 'p100': 0, 'map': 0},
 'alt.atheism': {'p10': 0.7188118811881188,
  'p20': 0.657920792079208,
  'p50': 0.5792079207920792,
  'p100': 0.4873267326732673,
  'map': 0.22119225647265803},
 'comp.graphics': {'p10': 0.505,
  'p20': 0.441,
  'p50': 0.412,
  'p100': 0.3898,
  'map': 0.19187835679356377},
 'comp.os.ms-windows.misc': {'p10': 0.675,
  'p20': 0.6115,
  'p50': 0.5736,
  'p100': 0.5822,
  'map': 0.4248298080553348},
 'comp.sys.ibm.pc.hardware': {'p10': 0.55,
  'p20': 0.452,
  'p50': 0.3778,
  'p100': 0.3629,
  'map': 0.24468762317678025},
 'comp.sys.mac.hardware': {'p10': 0.601,
  'p20': 0.505,
  'p50': 0.436,
  'p100': 0.4139,
  'map': 0.2860355930567415},
 'comp.windows.x': {'p10': 0.543,
  'p20': 0.4675,
  'p50': 0.3898,
  'p100': 0.3479,
  'map': 0.15858726830745754},
 'misc.forsale': {'p10': 0.477,
  'p20': 0.409,
  'p50': 0.3898,
  'p100': 0.3967,
  'map': 0.3125999886875747},
 'rec.autos': {'p10': 0.633,
  'p20': 0.5425,
  'p

In [162]:
import json
with open('results/data_lemma_tfidf_lib.json', 'w') as f:
    json.dump(result_final, f)

In [164]:
import pandas as pd

df = pd.DataFrame(result_final)

In [ ]:
df

,mini_newsgroups,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,...,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
p10,0,0.289109,0.635000,0.124000,0.230000,0.28400,0.544000,0.167000,0.237000,0.256000,...,0.401000,0.408000,0.234000,0.328000,0.40700,0.334000,0.244000,0.64400,0.417000,0.206061
p20,0,0.249010,0.486000,0.114000,0.168000,0.19150,0.451500,0.114000,0.182500,0.189500,...,0.331000,0.350000,0.166000,0.278500,0.32650,0.304000,0.212500,0.57300,0.392000,0.178283
p50,0,0.229703,0.293600,0.124800,0.139800,0.12060,0.331800,0.083800,0.156200,0.153800,...,0.277000,0.306600,0.105000,0.245400,0.28360,0.297000,0.191600,0.50320,0.330400,0.152323
p100,0,0.242673,0.207200,0.157600,0.149800,0.11390,0.250700,0.085700,0.164400,0.141800,...,0.276200,0.303100,0.093000,0.236200,0.28080,0.320500,0.199700,0.48520,0.299400,0.163030
map,0,0.140509,0.123349,0.125516,0.121173,0.11715,0.117307,0.116856,0.116644,0.115331,...,0.122767,0.129178,0.125166,0.123782,0.12532,0.142284,0.142529,0.14772,0.147294,0.145951


In [165]:
df =df.transpose()

In [166]:
df.drop('mini_newsgroups')

,p10,p20,p50,p100,map
alt.atheism,0.718812,0.657921,0.579208,0.487327,0.221192
comp.graphics,0.505000,0.441000,0.412000,0.389800,0.191878
comp.os.ms-windows.misc,0.675000,0.611500,0.573600,0.582200,0.424830
comp.sys.ibm.pc.hardware,0.550000,0.452000,0.377800,0.362900,0.244688
comp.sys.mac.hardware,0.601000,0.505000,0.436000,0.413900,0.286036
comp.windows.x,0.543000,0.467500,0.389800,0.347900,0.158587
misc.forsale,0.477000,0.409000,0.389800,0.396700,0.312600
rec.autos,0.633000,0.542500,0.450000,0.412300,0.237742
rec.motorcycles,0.726000,0.618000,0.478400,0.407100,0.205094
rec.sport.baseball,0.697000,0.601000,0.490800,0.437400,0.260773


In [167]:
df.describe()

,p10,p20,p50,p100,map
count,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.647934,0.574357,0.504238,0.466056,0.289204
std,0.192202,0.189009,0.186173,0.185786,0.170439
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.579798,0.482500,0.412000,0.389800,0.205094
50%,0.675000,0.601000,0.478400,0.437100,0.253688
75%,0.761000,0.657921,0.579208,0.550800,0.340091
max,0.941000,0.934500,0.926800,0.922100,0.877714
